In [ ]:
"""
Created on Mon Mar 04 17:32 2024

Apply weights to long historical period

Author: @claraburgard

"""

In [ ]:
import xarray as xr
import numpy as np
from tqdm.notebook import tqdm
import seaborn as sns
import multimelt.useful_functions as uf

In [ ]:
sns.set_context('paper')

In [ ]:
%matplotlib qt5

READ IN DATA

In [ ]:
home_path = '/bettik/burgardc/'
plot_path = '/bettik/burgardc/PLOTS/summer_paper_plots/'
outputpath_GL = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/GL_FLUX/'
inputpath_weights = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/ANALYSIS/'
inputpath_atmo = '/bettik/burgardc/DATA/SUMMER_PAPER/raw/TS_SMB_DATA/out/'


In [ ]:
inputpath_mask = home_path+'/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'
file_isf_orig = xr.open_dataset(inputpath_mask+'BedMachinev2_4km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf)

In [ ]:
sorted_isf_rignot = [11,69,43,28,12,57,
                     70,44,29,13,58,71,45,30,14,
                     59,72,46,
                     31,
                     15,61,73,47,32,16,48,33,17,62,49,34,18,63,74,
                     50,35,19,64,
                     10,
                     36,20,65,51,37,
                     22,38,52,23,66,53,39,24,
                     67,40,54,75,25,41,
                     26,42,55,68,60,27]

In [ ]:
param_classic_list = ['linear_local',
              'quadratic_local','quadratic_local_locslope',
              'quadratic_mixed_mean','quadratic_mixed_locslope',
              'lazero19',
              'boxes_4_pismyes_picopno']

param_NN_list = ['NNsmall']

CALVING

In [ ]:
inputpath_davison = home_path+'/DATA/SUMMER_PAPER/raw/'
davison_file = xr.open_dataset(inputpath_davison + 'steadystate_davison23.nc')
calving_flux = davison_file['calving_obs']

ABUMIP GL FLUX

In [ ]:
GL_flux = xr.open_dataset(outputpath_GL + 'all_GL_fluxes_varying_m.nc')

In [ ]:
GL_flux_ag = xr.Dataset()
GL_flux_ag['flux_Gt_ref'] = xr.concat([GL_flux['flux_Gt_ref_m1'].assign_coords({'m': 1}),
                                       GL_flux['flux_Gt_ref_m3'].assign_coords({'m': 3}),
                                       GL_flux['flux_Gt_ref_m5'].assign_coords({'m': 5})], dim='m')
GL_flux_ag['flux_Gt_ABUMIP'] = xr.concat([GL_flux['flux_Gt_ABUMIP_m1'].assign_coords({'m': 1}),
                                       GL_flux['flux_Gt_ABUMIP_m3'].assign_coords({'m': 3}),
                                       GL_flux['flux_Gt_ABUMIP_m5'].assign_coords({'m': 5})], dim='m')

SMB

In [ ]:
def start_end_year(mod, scen):
    if mod in ['CNRM-CM6-1','CNRM-ESM2-1']:
        ens_run = 'r1i1p1f2'
        to2300 = False
    elif mod in ['GISS-E2-1-H']:
        ens_run = 'r1i1p1f2'
        to2300 = True
    elif mod in ['ACCESS-CM2','ACCESS-ESM1-5','CESM2-WACCM','CanESM5','IPSL-CM6A-LR','MRI-ESM2-0']:
        ens_run = 'r1i1p1f1'
        to2300 = True
    elif mod in ['MPI-ESM1-2-HR','GFDL-CM4','GFDL-ESM4']:
        ens_run = 'r1i1p1f1'
        to2300 = False
    elif mod == 'UKESM1-0-LL':
        ens_run = 'r4i1p1f2'
        to2300 = True     
    elif mod == 'CESM2':
        ens_run = 'r11i1p1f1'
        to2300 = False        


    if scenario == 'historical':
        yystart = 1850 #1850
        yyend = 2014
    elif scenario == 'ssp245':
        yystart = 2015
        yyend = 2100  
    else:
        if to2300:
            yystart = 2015
            yyend = 2299
        else:
            yystart = 2015
            yyend = 2100  
    return ens_run, yystart, yyend

In [ ]:
melt_atmo_list2 = []
inputpath_atmo_Chris = '/bettik/burgardc/DATA/SUMMER_PAPER/raw/TS_SMB_DATA/out/'
inputpath_atmo_Nico = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/SMB_EMULATED/'
#for mod in ['ACCESS-ESM1-5','ACCESS-CM2','CESM2','CESM2-WACCM','CNRM-CM6-1','CNRM-ESM2-1',
#            'CanESM5','GFDL-CM4','GFDL-ESM4','GISS-E2-1-H', 'IPSL-CM6A-LR',
#           'MPI-ESM1-2-HR','MRI-ESM2-0','UKESM1-0-LL']:
    
melt_atmo3_list = []
for scenario in ['historical','ssp126','ssp245','ssp585']:

    melt_atmo_list = []
    for mod in ['CESM2','IPSL-CM6A-LR','MPI-ESM1-2-HR','UKESM1-0-LL']: #'CNRM-CM6-1',
        
        _, yystart, yyend = start_end_year(mod, scenario)
        #print(mod,yystart,yyend)
        
        if (scenario == 'historical') and (yystart == 1850):
            
            melt_atmo_1850 = xr.open_dataset(inputpath_atmo_Nico+mod+'_SMB_'+scenario+'-1850-1979.nc').sel(Nisf=rignot_isf)
            
            if (os.path.exists(inputpath_atmo_Chris+mod+'_SMB_'+scenario+'-1980-2014.nc')):
                melt_atmo = xr.open_dataset(inputpath_atmo_Chris+mod+'_SMB_'+scenario+'-1980-2014.nc').sel(Nisf=rignot_isf)
                melt_atmo['time'] = melt_atmo['time'].dt.year
            else:
                melt_atmo = xr.open_dataset(inputpath_atmo_Nico+mod+'_SMB_'+scenario+'-1980-2014.nc').sel(Nisf=rignot_isf)
                
            melt_atmo_list.append(xr.concat([melt_atmo_1850['SMB'],melt_atmo['SMB']], dim='time').assign_coords({'model': mod}))

        else:
            if os.path.exists(inputpath_atmo_Chris+mod+'_SMB_'+scenario+'-'+str(yystart)+'-'+str(yyend)+'.nc'):
                melt_atmo = xr.open_dataset(inputpath_atmo_Chris+mod+'_SMB_'+scenario+'-'+str(yystart)+'-'+str(yyend)+'.nc').sel(Nisf=rignot_isf)
                melt_atmo['time'] = melt_atmo['time'].dt.year
                melt_atmo_list.append(melt_atmo['SMB'].assign_coords({'model': mod}))
            else:
                melt_atmo = xr.open_dataset(inputpath_atmo_Nico+mod+'_SMB_'+scenario+'-'+str(yystart)+'-'+str(yyend)+'.nc').sel(Nisf=rignot_isf)
                melt_atmo_list.append(melt_atmo['SMB'].assign_coords({'model': mod}))


    for mod in ['ACCESS-ESM1-5','ACCESS-CM2','CESM2-WACCM','CNRM-CM6-1','CNRM-ESM2-1',
                'CanESM5','GFDL-CM4','GFDL-ESM4','GISS-E2-1-H', 
               'MRI-ESM2-0']:

        _, yystart, yyend = start_end_year(mod, scenario)
        #print(mod,yystart,yyend)
        
        if mod == 'GFDL-CM4' and scenario == 'ssp126':
            melt_atmo = melt_atmo * np.nan
        
        else:
            
            if (scenario == 'historical') and (yystart == 1850):
                    
                melt_atmo_1850 = xr.open_dataset(inputpath_atmo_Nico+mod+'_SMB_'+scenario+'-1850-1979.nc').sel(Nisf=rignot_isf)
                melt_atmo = xr.open_dataset(inputpath_atmo_Nico+mod+'_SMB_'+scenario+'-1980-2014.nc').sel(Nisf=rignot_isf)
                melt_atmo_list.append(xr.concat([melt_atmo_1850['SMB'],melt_atmo['SMB']], dim='time').assign_coords({'model': mod}))
            
            else:
            
                melt_atmo = xr.open_dataset(inputpath_atmo_Nico+mod+'_SMB_'+scenario+'-'+str(yystart)+'-'+str(yyend)+'.nc').sel(Nisf=rignot_isf)
                melt_atmo_list.append(melt_atmo['SMB'].assign_coords({'model': mod}))

    melt_atmo_scenario = xr.concat(melt_atmo_list, dim='model')
    melt_atmo3_list.append(melt_atmo_scenario.assign_coords({'scenario': scenario}))
melt_atmo_all = xr.concat(melt_atmo3_list, dim='scenario').sel(Nisf=rignot_isf)
#melt_atmo_all = xr.concat(melt_atmo_list2, dim='model')


OCEAN

In [ ]:
Gt_list_models = []
box1_list_models = []

for mod in tqdm(['ACCESS-ESM1-5','ACCESS-CM2','CESM2','CESM2-WACCM','CNRM-CM6-1','CNRM-ESM2-1',
            'CanESM5','GFDL-CM4','GFDL-ESM4','GISS-E2-1-H', 'IPSL-CM6A-LR',
           'MPI-ESM1-2-HR','MRI-ESM2-0','UKESM1-0-LL']): #
    
    outputpath_melt = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/OCEAN_MELT_RATE_CMIP/'+mod+'/'

    melt1D_list1 = []

    for mparam in param_classic_list:

        melt1D_scenario_list = []
        for scenario in ['historical','ssp126','ssp245','ssp585']:
            if mod == 'GFDL-CM4' and scenario == 'ssp126':
                melt_1D = melt_1D * np.nan 
            else:
                melt_1D = xr.open_dataset(outputpath_melt + 'eval_metrics_1D_'+mparam+'_'+scenario+'_oneFRIS_anoISMIP.nc')
                melt1D_scenario_list.append(melt_1D.assign_coords({'scenario': scenario}))
        melt1D_scenario = xr.concat(melt1D_scenario_list, dim='scenario')
        melt1D_list1.append(melt1D_scenario.assign_coords({'param':mparam}))
    melt1D_classic = xr.concat(melt1D_list1, dim='param')       
    Gt_classic = melt1D_classic['melt_1D_Gt_per_y']
    box1_classic = melt1D_classic['melt_1D_mean_myr_box1']
    
    melt1D_list2 = []
    for mparam in param_NN_list:

        melt1D_scenario_list = []
        for scenario in ['historical','ssp126','ssp245','ssp585']:
            if mod == 'GFDL-CM4' and scenario == 'ssp126':
                melt_1D = melt_1D * np.nan 
            else:
                melt_1D = xr.open_dataset(outputpath_melt + 'evalmetrics_1D_'+mparam+'_'+scenario+'_anoISMIP.nc')
                melt1D_scenario_list.append(melt_1D.assign_coords({'scenario': scenario}))
        melt1D_scenario = xr.concat(melt1D_scenario_list, dim='scenario')
        melt1D_list2.append(melt1D_scenario.assign_coords({'param':mparam}))
    melt1D_NN = xr.concat(melt1D_list2, dim='param')   
    Gt_NN = melt1D_NN['predicted_melt'].sel(metrics='Gt')
    box1_NN = melt1D_NN['predicted_melt'].sel(metrics='box1')
    
    Gt_all = xr.concat([Gt_classic, Gt_NN], dim='param')
    box1_all = xr.concat([box1_classic, box1_NN], dim='param')
    
    Gt_list_models.append(Gt_all.assign_coords({'model':mod}))
    box1_list_models.append(box1_all.assign_coords({'model':mod}))

Gt_all_mod_anoISMIP = xr.concat(Gt_list_models, dim='model')
box1_all_mod_anoISMIP = xr.concat(box1_list_models, dim='model')


    
    

In [ ]:
Gt_list_models = []
box1_list_models = []

for mod in tqdm(['ACCESS-ESM1-5','ACCESS-CM2','CESM2','CESM2-WACCM','CNRM-CM6-1','CNRM-ESM2-1',
            'CanESM5','GFDL-CM4','GFDL-ESM4','GISS-E2-1-H', 'IPSL-CM6A-LR',
           'MPI-ESM1-2-HR','MRI-ESM2-0','UKESM1-0-LL']): #
    
    outputpath_melt = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/OCEAN_MELT_RATE_CMIP/'+mod+'/'

    melt1D_list1 = []

    for mparam in param_classic_list:

        melt1D_scenario_list = []
        for scenario in ['historical','ssp126','ssp245','ssp585']:
            if mod == 'GFDL-CM4' and scenario == 'ssp126':
                melt_1D = melt_1D * np.nan 
            else:
                melt_1D = xr.open_dataset(outputpath_melt + 'eval_metrics_1D_'+mparam+'_'+scenario+'_oneFRIS_anoNEMO.nc')
                melt1D_scenario_list.append(melt_1D.assign_coords({'scenario': scenario}))
        melt1D_scenario = xr.concat(melt1D_scenario_list, dim='scenario')
        melt1D_list1.append(melt1D_scenario.assign_coords({'param':mparam}))
    melt1D_classic = xr.concat(melt1D_list1, dim='param')       
    Gt_classic = melt1D_classic['melt_1D_Gt_per_y']
    box1_classic = melt1D_classic['melt_1D_mean_myr_box1']
    
    melt1D_list2 = []
    for mparam in param_NN_list:

        melt1D_scenario_list = []
        for scenario in ['historical','ssp126','ssp245','ssp585']:
            if mod == 'GFDL-CM4' and scenario == 'ssp126':
                melt_1D = melt_1D * np.nan 
            else:
                melt_1D = xr.open_dataset(outputpath_melt + 'evalmetrics_1D_'+mparam+'_'+scenario+'_anoNEMO.nc')
                melt1D_scenario_list.append(melt_1D.assign_coords({'scenario': scenario}))
        melt1D_scenario = xr.concat(melt1D_scenario_list, dim='scenario')
        melt1D_list2.append(melt1D_scenario.assign_coords({'param':mparam}))
    melt1D_NN = xr.concat(melt1D_list2, dim='param')   
    Gt_NN = melt1D_NN['predicted_melt'].sel(metrics='Gt')
    box1_NN = melt1D_NN['predicted_melt'].sel(metrics='box1')
    
    Gt_all = xr.concat([Gt_classic, Gt_NN], dim='param')
    box1_all = xr.concat([box1_classic, box1_NN], dim='param')
    
    Gt_list_models.append(Gt_all.assign_coords({'model':mod}))
    box1_list_models.append(box1_all.assign_coords({'model':mod}))

Gt_all_mod_anoNEMO = xr.concat(Gt_list_models, dim='model')
box1_all_mod_anoNEMO = xr.concat(box1_list_models, dim='model')


    
    

MASS BALANCE

In [ ]:
ano_and_weights = xr.open_dataset(inputpath_weights + 'NEMOorISMIP_choice_and_weights.nc')
ano_and_weights_2300 = xr.open_dataset(inputpath_weights + 'NEMOorISMIP_choice_and_weights_2300.nc')

In [ ]:
mass_balance_simu_list = []
for kisf in tqdm(file_isf.Nisf):
    if ano_and_weights['ano_choice'].sel(Nisf=kisf).values == 'NEMO':
        mass_balance_simu_list.append(Gt_all_mod_anoNEMO.sel(Nisf=kisf) - melt_atmo_all.sel(Nisf=kisf) + davison_file['calving_obs'].sel(Nisf=kisf) - GL_flux_ag['flux_Gt_ABUMIP'].sel(Nisf=kisf))
    elif ano_and_weights['ano_choice'].sel(Nisf=kisf).values == 'ISMIP':
        mass_balance_simu_list.append(Gt_all_mod_anoISMIP.sel(Nisf=kisf) - melt_atmo_all.sel(Nisf=kisf) + davison_file['calving_obs'].sel(Nisf=kisf) - GL_flux_ag['flux_Gt_ABUMIP'].sel(Nisf=kisf))
mass_balance_simu = xr.concat(mass_balance_simu_list, dim='Nisf')
        

In [ ]:
mass_balance_simu_2300 = mass_balance_simu.sel(model=['ACCESS-CM2','ACCESS-ESM1-5','CanESM5',
                                                        'CESM2-WACCM','GISS-E2-1-H', 'IPSL-CM6A-LR',
                                                        'MRI-ESM2-0','UKESM1-0-LL'])


In [ ]:
#bay_weights = xr.open_dataset(inputpath_weights + 'bayesian_weights_davison.nc')['bay_weights']
bay_weights = ano_and_weights['bay_weights']
bay_weights_2300 = ano_and_weights_2300['bay_weights']

In [ ]:
sens_weights = xr.DataArray(data=np.array([0.11,
                                           0.24,
                                           0.03,
                                           0.10,
                                           0.10,
                                           0.10,
                                           0.10,
                                           0.24,
                                           0.47,
                                           0.41,
                                           0.12,
                                           0.43,
                                           0.39,
                                           0.05]), dims=['model']).assign_coords({'model': 
                                                                                  ['ACCESS-CM2','ACCESS-ESM1-5','CanESM5',
                                                                                   'CESM2','CESM2-WACCM','CNRM-CM6-1','CNRM-ESM2-1',
                                                                                   'GFDL-CM4','GFDL-ESM4','GISS-E2-1-H', 'IPSL-CM6A-LR',
                                                                                   'MPI-ESM1-2-HR','MRI-ESM2-0','UKESM1-0-LL']})

In [ ]:
ensmean_weighted = uf.weighted_mean(mass_balance_simu, ['model','param','m'], bay_weights * sens_weights.sel(model=mass_balance_simu.model))
ensmean_weighted_2300 = uf.weighted_mean(mass_balance_simu_2300, ['model','param','m'], bay_weights_2300 * sens_weights.sel(model=mass_balance_simu_2300.model))

#ensmean_nonweighted = mass_balance_simu.mean(['model','param'])

In [ ]:
da_weighted = mass_balance_simu.weighted(bay_weights * sens_weights.sel(model=mass_balance_simu.model))
da_weighted_2300 = mass_balance_simu_2300.weighted(bay_weights_2300 * sens_weights.sel(model=mass_balance_simu_2300.model))

In [ ]:
#ensstat_weighted = da_weighted.quantile([0.05,0.5,0.95],dim=['model','param'])
ensstat_weighted = da_weighted.quantile([0.05,0.1,0.33,0.66,0.5,0.9,0.95],dim=['model','param','m'])
ensstat_weighted_2300 = da_weighted_2300.quantile([0.05,0.1,0.33,0.66,0.5,0.9,0.95],dim=['model','param','m'])


VIABILITY LIMIT

In [ ]:
limit_50 = ensstat_weighted.sel(quantile = 0.5) > 0
limit_66 = ensstat_weighted.sel(quantile = 0.33) > 0
limit_90 = ensstat_weighted.sel(quantile = 0.10) > 0

In [ ]:
limit_50_2300 = ensstat_weighted_2300.sel(quantile = 0.5) > 0
limit_66_2300 = ensstat_weighted_2300.sel(quantile = 0.33) > 0
limit_90_2300 = ensstat_weighted_2300.sel(quantile = 0.10) > 0

In [ ]:
fig, ax = plt.subplots()
fig1 = plt.gcf()
fig.set_size_inches(8.25, 8.25*1.5)

ll = 0
label_list = []

for scen in ['historical','ssp126','ssp245','ssp585']: 
    
    if scen == 'ssp126':
        ccol = 'gold'
        shift=+0.3
    elif scen == 'ssp245':
        ccol = 'darkorange'
        shift=0
    elif scen == 'ssp585':
        ccol = 'red'
        shift=-0.3
    elif scen == 'historical':
        ccol = 'grey'
        shift=0        
    

    yval = xr.DataArray(data=np.array(range(len(sorted_isf_rignot))), dims=['Nisf']).assign_coords({'Nisf': sorted_isf_rignot[::-1]})
    _ , yval_br = xr.broadcast(limit_66.time,yval)
    
    ll2=0
    #for k,kisf in enumerate(file_isf.Nisf.sel(Nisf=sorted_isf_rignot[::-1])):
    for k,kisf in enumerate(yval.Nisf):
        time_nonvia50 = limit_50.time.where(limit_50.sel(Nisf=kisf,scenario=scen))
        time_nonvia66 = limit_66.time.where(limit_66.sel(Nisf=kisf,scenario=scen))
        time_nonvia90 = limit_90.time.where(limit_90.sel(Nisf=kisf,scenario=scen))
        
        lim50_yes = limit_50.sel(Nisf=kisf,scenario=scen).astype(int)
        lim66_yes = limit_66.sel(Nisf=kisf,scenario=scen).astype(int)
        lim90_yes = limit_90.sel(Nisf=kisf,scenario=scen).astype(int)

        plt.scatter(time_nonvia50,yval_br.sel(Nisf=kisf)+shift+ll2,alpha=lim50_yes*0.33,s=10,edgecolors='None',color=ccol)
        plt.scatter(time_nonvia66,yval_br.sel(Nisf=kisf)+shift+ll2,alpha=lim66_yes*0.4,s=10,edgecolors='None',color=ccol)
        plt.scatter(time_nonvia90,yval_br.sel(Nisf=kisf)+shift+ll2,alpha=lim90_yes*1,s=10,edgecolors='None',color=ccol)
        
        
        if ll == 0:
            label_list.append(file_isf['isf_name'].sel(Nisf=kisf).values)
            
            if kisf in [69,59,48,50,22,40,41]:
                ll2 = ll2+1
                plt.scatter(time_nonvia66,yval_br.sel(Nisf=kisf)+shift*np.nan,alpha=lim66_yes*0.3,s=10,edgecolors='None',color=ccol)
                plt.axhline(yval_br.sel(Nisf=kisf).max().values+ll2,linestyle='--')
                label_list.append(' ')
                
        
        else:
            if kisf in [69,59,48,50,22,40,41]:                
                ll2 = ll2+1
        
    ll = ll+1
    
ax.set_yticks(np.arange(len(label_list)), labels=label_list)    
#ax.set_yticks(yval, labels=file_isf['isf_name'].sel(Nisf=sorted_isf_rignot[::-1]).values)
#ax.set_yticklabels(file_isf['isf_name'].sel(Nisf=sorted_isf_rignot).values)
ax.set_xlim(1850,2100)
#ax.set_title(scen)

sns.despine()
plt.grid(linestyle='--')
plt.tight_layout()

fig.savefig(plot_path + 'scatterbarplots_allscen_until2100.png')


UNTIL 2300

In [ ]:
fig, ax = plt.subplots()
fig1 = plt.gcf()
fig.set_size_inches(8.25, 8.25*1.5)

ll = 0
label_list = []

for scen in ['historical','ssp126','ssp585']: 
    
    if scen == 'ssp126':
        ccol = 'gold'
        shift=+0.2
    elif scen == 'ssp245':
        ccol = 'darkorange'
        shift=0
    elif scen == 'ssp585':
        ccol = 'red'
        shift=-0.2
    elif scen == 'historical':
        ccol = 'grey'
        shift=0        
    

    yval = xr.DataArray(data=np.array(range(len(sorted_isf_rignot))), dims=['Nisf']).assign_coords({'Nisf': sorted_isf_rignot[::-1]})
    _ , yval_br = xr.broadcast(limit_66.time,yval)
    
    ll2=0
    #for k,kisf in enumerate(file_isf.Nisf.sel(Nisf=sorted_isf_rignot[::-1])):
    for k,kisf in enumerate(yval.Nisf):
        time_nonvia50 = limit_50_2300.time.where(limit_50_2300.sel(Nisf=kisf,scenario=scen))
        time_nonvia66 = limit_66_2300.time.where(limit_66_2300.sel(Nisf=kisf,scenario=scen))
        time_nonvia90 = limit_90_2300.time.where(limit_90_2300.sel(Nisf=kisf,scenario=scen))

        lim50_yes = limit_50_2300.sel(Nisf=kisf,scenario=scen).astype(int)
        lim66_yes = limit_66_2300.sel(Nisf=kisf,scenario=scen).astype(int)
        lim90_yes = limit_90_2300.sel(Nisf=kisf,scenario=scen).astype(int)

        plt.scatter(time_nonvia50,yval_br.sel(Nisf=kisf)+shift+ll2,alpha=lim50_yes*0.1,s=10,edgecolors='None',color=ccol)
        plt.scatter(time_nonvia66,yval_br.sel(Nisf=kisf)+shift+ll2,alpha=lim66_yes*0.2,s=10,edgecolors='None',color=ccol)
        plt.scatter(time_nonvia90,yval_br.sel(Nisf=kisf)+shift+ll2,alpha=lim90_yes*1,s=10,edgecolors='None',color=ccol)
        
        
        if ll == 0:
            label_list.append(file_isf['isf_name'].sel(Nisf=kisf).values)
            
            if kisf in [69,59,48,50,22,40,41]:
                ll2 = ll2+1
                plt.scatter(time_nonvia66,yval_br.sel(Nisf=kisf)+shift*np.nan,alpha=lim66_yes*0.3,s=10,edgecolors='None',color=ccol)
                plt.axhline(yval_br.sel(Nisf=kisf).max().values+ll2,linestyle='--')
                label_list.append(' ')
                
        
        else:
            if kisf in [69,59,48,50,22,40,41]:                
                ll2 = ll2+1
        
    ll = ll+1
    
ax.set_yticks(np.arange(len(label_list)), labels=label_list)    
#ax.set_yticks(yval, labels=file_isf['isf_name'].sel(Nisf=sorted_isf_rignot[::-1]).values)
#ax.set_yticklabels(file_isf['isf_name'].sel(Nisf=sorted_isf_rignot).values)
ax.set_xlim(1850,2300)
#ax.set_title(scen)

sns.despine()
plt.grid(linestyle='--')
plt.tight_layout()

fig.savefig(plot_path + 'scatterbarplots_allscen_until2300.png')


PROBABILITY CURVES

In [ ]:
all_weights = bay_weights * sens_weights.sel(model=mass_balance_simu.model)
non_viable_mask = mass_balance_simu > 0
non_viable_sum = all_weights.where(non_viable_mask).sum(['model','param','m'])
all_weights_sum = all_weights.sum(['model','param','m'])
non_viable_normed = non_viable_sum/all_weights_sum

Define viability limit as a date

In [ ]:
viability_50_diff = limit_50.sel(time=range(2100,2014,-1)).cumsum('time').diff('time')
limit50_max = viability_50_diff.time.where(viability_50_diff == 0).max('time') + 1
limit50_max = limit50_max.where(limit50_max<2100)
limit50_max = limit50_max.where(~(np.isnan(limit50_max) & limit_50.sel(time=2015)), 2014)

viability_66_diff = limit_66.sel(time=range(2100,2014,-1)).cumsum('time').diff('time')
limit66_max = viability_66_diff.time.where(viability_66_diff == 0).max('time') + 1
limit66_max = limit66_max.where(limit66_max<2100)
limit66_max = limit66_max.where(~(np.isnan(limit66_max) & limit_66.sel(time=2015)), 2014)

viability_90_diff = limit_90.sel(time=range(2100,2014,-1)).cumsum('time').diff('time')
limit90_max = viability_90_diff.time.where(viability_90_diff == 0).max('time') + 1
limit90_max = limit90_max.where(limit90_max<2100)
limit90_max = limit90_max.where(~(np.isnan(limit90_max) & limit_90.sel(time=2015)), 2014)

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
f.set_size_inches(8.25*3, 8.25*3)

ax={}

leg_hdl = []

i = 0


for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)

    ax[i].plot(non_viable_normed.time.sel(time=range(1850,2015)), non_viable_normed.sel(Nisf=kisf,scenario='historical',time=range(1850,2015)), color='grey')    
    ax[i].plot(non_viable_normed.time.sel(time=range(2015,2100)), non_viable_normed.sel(Nisf=kisf,scenario='ssp126',time=range(2015,2100)), color='gold')
    ax[i].plot(non_viable_normed.time.sel(time=range(2015,2100)), non_viable_normed.sel(Nisf=kisf,scenario='ssp245',time=range(2015,2100)), color='darkorange')
    ax[i].plot(non_viable_normed.time.sel(time=range(2015,2100)), non_viable_normed.sel(Nisf=kisf,scenario='ssp585',time=range(2015,2100)), color='red')
    ax[i].axhline(y=0.50, color='deepskyblue', alpha=0.9, linestyle='--')
    ax[i].axhline(y=0.66, color='deepskyblue', alpha=0.5, linestyle='--')
    ax[i].axhline(y=0.90, color='deepskyblue', alpha=0.2, linestyle='--')
    ax[i].axvline(x=limit50_max.sel(Nisf=kisf, scenario='ssp585'), color='black', linestyle='--')

    #ax[i].set_title(ano_choice.sel(Nisf=kisf))

    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:#
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf['isf_name'].sel(Nisf=kisf).values)+' '+str(kisf))
    #if file_isf['isf_name'].sel(Nisf=kisf).values not in ['Larsen B', 'Wordie']:
    #    ax[i].set_ylim(0,mass_balance_simu_anoISMIP.sel(Nisf=kisf).max().values)
    ax[i].set_xlim(1850,2099)
    ax[i].set_ylim(0,1)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
f.savefig(plot_path + 'prob_distrib_allscen_until2100.pdf')


In [ ]:
all_weights_2300 = bay_weights_2300 * sens_weights.sel(model=mass_balance_simu_2300.model)
non_viable_mask_2300 = mass_balance_simu_2300 > 0
non_viable_sum_2300 = all_weights_2300.where(non_viable_mask_2300).sum(['model','param','m'])
all_weights_sum_2300 = all_weights_2300.sum(['model','param','m'])
non_viable_normed_2300 = non_viable_sum_2300/all_weights_sum_2300

Define viability limit as a date

In [ ]:
viability_50_diff = limit_50_2300.sel(time=range(2299,2014,-1)).cumsum('time').diff('time')
limit50_max_2300 = viability_50_diff.time.where(viability_50_diff == 0).max('time') + 1
limit50_max_2300 = limit50_max_2300.where(limit50_max_2300<2299)
limit50_max_2300 = limit50_max_2300.where(~(np.isnan(limit50_max_2300) & limit_50_2300.sel(time=2015)), 2014)

viability_66_diff = limit_66_2300.sel(time=range(2299,2014,-1)).cumsum('time').diff('time')
limit66_max_2300 = viability_66_diff.time.where(viability_66_diff == 0).max('time') + 1
limit66_max_2300 = limit66_max_2300.where(limit66_max_2300<2299)
limit66_max_2300 = limit66_max_2300.where(~(np.isnan(limit66_max_2300) & limit_66_2300.sel(time=2015)), 2014)

viability_90_diff = limit_90_2300.sel(time=range(2299,2014,-1)).cumsum('time').diff('time')
limit90_max_2300 = viability_90_diff.time.where(viability_90_diff == 0).max('time') + 1
limit90_max_2300 = limit90_max_2300.where(limit90_max_2300<2299)
limit90_max_2300 = limit90_max_2300.where(~(np.isnan(limit90_max_2300) & limit_90_2300.sel(time=2015)), 2014)

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
f.set_size_inches(8.25*3, 8.25*3)

ax={}

leg_hdl = []

i = 0


for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)
    
    
    ax[i].plot(non_viable_normed_2300.time.sel(time=range(1850,2015)), non_viable_normed_2300.sel(Nisf=kisf,scenario='historical',time=range(1850,2015)), color='grey')    
    ax[i].plot(non_viable_normed_2300.time.sel(time=range(2015,2300)), non_viable_normed_2300.sel(Nisf=kisf,scenario='ssp126',time=range(2015,2300)), color='gold')
    ax[i].plot(non_viable_normed_2300.time.sel(time=range(2015,2300)), non_viable_normed_2300.sel(Nisf=kisf,scenario='ssp585',time=range(2015,2300)), color='red')
    ax[i].axhline(y=0.50, color='deepskyblue', alpha=0.9, linestyle='--')
    ax[i].axhline(y=0.66, color='deepskyblue', alpha=0.5, linestyle='--')
    ax[i].axhline(y=0.90, color='deepskyblue', alpha=0.2, linestyle='--')
    ax[i].axvline(x=limit66_max_2300.sel(Nisf=kisf, scenario='ssp585'), color='black', linestyle='--')

    #ax[i].set_title(ano_choice.sel(Nisf=kisf))

    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:#
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf['isf_name'].sel(Nisf=kisf).values)+' '+str(kisf))
    #if file_isf['isf_name'].sel(Nisf=kisf).values not in ['Larsen B', 'Wordie']:
    #    ax[i].set_ylim(0,mass_balance_simu_anoISMIP.sel(Nisf=kisf).max().values)
    ax[i].set_xlim(2015,2300)
    ax[i].set_ylim(0,1)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()

f.savefig(plot_path + 'prob_distrib_allscen_until2300.pdf')


In [ ]:
viability_limit_dates = xr.Dataset()
viability_limit_dates['viability_limit_2100'] = xr.concat([limit50_max.assign_coords({'quant': 50}),
                                                          limit66_max.assign_coords({'quant': 66}),
                                                          limit90_max.assign_coords({'quant': 90})], dim='quant')
viability_limit_dates['viability_limit_2300'] = xr.concat([limit50_max_2300.assign_coords({'quant': 50}),
                                                          limit66_max_2300.assign_coords({'quant': 66}),
                                                          limit90_max_2300.assign_coords({'quant': 90})], dim='quant')
viability_limit_dates.to_netcdf(inputpath_weights + 'viability_limits.nc')

VIABILITY LIMIT (this was dividing the m's, not needed anymore)

In [ ]:
limit_dates = xr.Dataset()

# 66 % above lowest ABUMIP limit
limit_dates['lowest_66'] = ensstat_weighted.sel(quantile=0.66) > 0

# 90 % above lowest ABUMIP limit
limit_dates['lowest_90'] = ensstat_weighted.sel(quantile=0.90) > GL_flux_ABUMIP_inone.min('m')

# 66 % above highest ABUMIP limit
limit_dates['highest_66'] = ensstat_weighted.sel(quantile=0.66) > GL_flux_ABUMIP_inone.max('m')

# 90 % above highest ABUMIP limit
limit_dates['highest_90'] = ensstat_weighted.sel(quantile=0.90) > GL_flux_ABUMIP_inone.max('m')

In [ ]:
kisf=66
scen = 'ssp585'

limit_dates['highest_66'].sel(scenario='ssp585', Nisf=kisf).plot()

In [ ]:
for stat in ['lowest_66','highest_66','lowest_90','highest_90']:
    fyear = limit_dates[stat].cumsum('time')
    time_br, _ = xr.broadcast(limit_dates.time, limit_dates)
    limit_dates[stat+'_threshold'] = time_br.where(fyear == 1).max('time')
    limit_dates[stat+'_threshold'] = limit_dates[stat+'_threshold'].where(np.isfinite(limit_dates[stat+'_threshold']),2014)

In [ ]:
width = xr.Dataset()
#for scen in ['historical','ssp126','ssp245','ssp585']:
    #allmax = limit_dates['highest_66_threshold'].where(limit_dates['highest_90_threshold'] <= limit_dates['highest_66_threshold'], limit_dates['highest_90_threshold'])
    #allmin = limit_dates['lowest_66_threshold'].where(limit_dates['lowest_90_threshold'] >= limit_dates['lowest_66_threshold'], limit_dates['lowest_90_threshold'])
width['range66'] = limit_dates['highest_66_threshold'] - limit_dates['lowest_66_threshold']
width['range90'] = limit_dates['highest_90_threshold'] - limit_dates['lowest_90_threshold']
    #'historical',

In [ ]:
scen='ssp585'
kisf = 66

limit_dates_list = []
for stat in ['lowest_66','highest_66','lowest_90','highest_90']:
    print(stat)
    limit_dates_scen_list = []
    for scen in ['historical','ssp126','ssp245','ssp585']:
        if scen == 'historical':
            limit_dates[stat] = 1850 + (limit_dates[stat+'_threshold'].sel(scenario=scen) == False).sum('time')
        else:
            limit_dates[stat] = 2014 + (limit_dates[stat+'_threshold'].sel(scenario=scen) == False).sum('time')
        limit_dates_scen_list.append(limit_dates[stat].assign_coords({'scenario': scen}))
    limit_dates_scen = xr.concat(limit_dates_scen_list, dim='scenario')
    limit_dates_list.append(limit_dates_scen)
limit_dates_all = xr.merge(limit_dates_list)




In [ ]:
ensstat_weighted['quantile']

In [ ]:
width_plot['historical_66'].sel(Nisf=kisf)

In [ ]:
for scen in ['ssp126','ssp245','ssp585']:
    
    if scen == 'ssp126':
        ccol = 'gold'
    elif scen == 'ssp245':
        ccol = 'darkorange'
    elif scen == 'ssp585':
        ccol = 'red'
    
    fig, ax = plt.subplots()
    fig1 = plt.gcf()
    fig.set_size_inches(8.25, 8.25*1.5)

    yval = np.array(range(len(sorted_isf_rignot)))
    width_plot = width.where(np.isfinite(width), 0)

    for k,kisf in enumerate(file_isf.Nisf.sel(Nisf=sorted_isf_rignot[::-1])):
        ax.barh(k, width=width_plot['range66'].sel(Nisf=kisf,scenario=scen), left=limit_dates['lowest_66_threshold'].sel(Nisf=kisf,scenario=scen), height=0.35, color=ccol,alpha=0.4)
        ax.barh(k, width=width_plot['range90'].sel(Nisf=kisf,scenario=scen), left=limit_dates['lowest_90_threshold'].sel(Nisf=kisf,scenario=scen), height=0.35, color=ccol,alpha=0.4)#,yerr=d["{0}_per1_all_std".format(var)][4,:])
        #ax.barh(k, width=2300-limit_dates['highest_90_threshold'].sel(Nisf=kisf,scenario=scen), left=limit_dates['highest_90_threshold'].sel(Nisf=kisf,scenario=scen), height=0.4, color='darkgrey',hatch='/',alpha=0.9)
        
    ax.set_yticks(yval, labels=file_isf['isf_name'].sel(Nisf=sorted_isf_rignot[::-1]).values)
    #ax.set_yticklabels(file_isf['isf_name'].sel(Nisf=sorted_isf_rignot).values)
    ax.set_xlim(2014,2100)
    ax.set_title(scen)

    sns.despine()
    plt.grid(linestyle='--')
    plt.tight_layout()
    
    fig.savefig(plot_path + 'barplots_'+scen+'_until2100.png')


In [ ]:
for scen in ['ssp126','ssp245','ssp585']:
    
    if scen == 'ssp126':
        ccol = 'gold'
    elif scen == 'ssp245':
        ccol = 'darkorange'
    elif scen == 'ssp585':
        ccol = 'red'
    
    fig, ax = plt.subplots()
    fig1 = plt.gcf()
    fig.set_size_inches(8.25, 8.25*1.5)

    yval = np.array(range(len(sorted_isf_rignot)))
    width_plot = width.where(np.isfinite(width), 0)

    for k,kisf in enumerate(file_isf.Nisf.sel(Nisf=sorted_isf_rignot[::-1])):
        ax.barh(k, width=width_plot['range66'].sel(Nisf=kisf,scenario=scen), left=limit_dates['lowest_66_threshold'].sel(Nisf=kisf,scenario=scen), height=0.8, color=ccol,alpha=0.4)
        ax.barh(k, width=width_plot['range90'].sel(Nisf=kisf,scenario=scen), left=limit_dates['lowest_90_threshold'].sel(Nisf=kisf,scenario=scen), height=0.8, color=ccol,alpha=0.4)#,yerr=d["{0}_per1_all_std".format(var)][4,:])

    ax.set_yticks(yval, labels=file_isf['isf_name'].sel(Nisf=sorted_isf_rignot[::-1]).values)
    #ax.set_yticklabels(file_isf['isf_name'].sel(Nisf=sorted_isf_rignot).values)
    ax.set_xlim(2014,2300)
    ax.set_title(scen)

    sns.despine()
    plt.grid(linestyle='--')
    plt.tight_layout()
    
    
    fig.savefig(plot_path + 'barplots_'+scen+'_until2300.png')


In [ ]:
scen = 'historical'
ccol = 'darkgrey'


fig, ax = plt.subplots()
fig1 = plt.gcf()
fig.set_size_inches(8.25, 8.25*1.5)

yval = np.array(range(len(sorted_isf_rignot)))
width_plot = width.where(np.isfinite(width), 0)

for k,kisf in enumerate(file_isf.Nisf.sel(Nisf=sorted_isf_rignot[::-1])):
    ax.barh(k, width=width_plot['range66'].sel(Nisf=kisf,scenario=scen), left=limit_dates['lowest_66_threshold'].sel(Nisf=kisf,scenario=scen), height=0.8, color=ccol,alpha=0.4)
    ax.barh(k, width=width_plot['range90'].sel(Nisf=kisf,scenario=scen), left=limit_dates['lowest_90_threshold'].sel(Nisf=kisf,scenario=scen), height=0.8, color=ccol,alpha=0.4)#,yerr=d["{0}_per1_all_std".format(var)][4,:])

ax.set_yticks(yval, labels=file_isf['isf_name'].sel(Nisf=sorted_isf_rignot[::-1]).values)
#ax.set_yticklabels(file_isf['isf_name'].sel(Nisf=sorted_isf_rignot).values)
ax.set_xlim(1850,2014)
ax.set_title(scen)

sns.despine()
plt.grid(linestyle='--')
plt.tight_layout()

fig.savefig(plot_path + 'barplots_'+scen+'.png')


In [ ]:
fig, ax = plt.subplots()
fig1 = plt.gcf()
fig.set_size_inches(8.25, 8.25*2)

yval = np.array(range(len(sorted_isf_rignot)))
width_plot = width.where(np.isfinite(width), 0)

for kisf in file_isf.Nisf.sel(Nisf=sorted_isf_rignot):
    scen = 'ssp126'
    ax.barh(kisf, width=width_plot['range66'].sel(Nisf=kisf,scenario=scen), left=limit_dates['lowest_66_threshold'].sel(Nisf=kisf,scenario=scen), height=0.2, color='gold',alpha=0.5)
    ax.barh(kisf, width=width_plot['range90'].sel(Nisf=kisf,scenario=scen), left=limit_dates['lowest_90_threshold'].sel(Nisf=kisf,scenario=scen), height=0.2, color='gold',alpha=0.5)#,yerr=d["{0}_per1_all_std".format(var)][4,:])
    scen = 'ssp245'
    ax.barh(kisf+0.25, width=width_plot['range66'].sel(Nisf=kisf,scenario=scen), left=limit_dates['lowest_66_threshold'].sel(Nisf=kisf,scenario=scen), height=0.2, color='darkorange',alpha=0.5)
    ax.barh(kisf+0.25, width=width_plot['range90'].sel(Nisf=kisf,scenario=scen), left=limit_dates['lowest_90_threshold'].sel(Nisf=kisf,scenario=scen), height=0.2, color='darkorange',alpha=0.5)#,yerr=d["{0}_per1_all_std".format(var)][4,:])
    scen = 'ssp585'
    ax.barh(kisf+0.5, width=width_plot['range66'].sel(Nisf=kisf,scenario=scen), left=limit_dates['lowest_66_threshold'].sel(Nisf=kisf,scenario=scen), height=0.2, color='red',alpha=0.5)
    ax.barh(kisf+0.5, width=width_plot['range90'].sel(Nisf=kisf,scenario=scen), left=limit_dates['lowest_90_threshold'].sel(Nisf=kisf,scenario=scen), height=0.2, color='red',alpha=0.5)#,yerr=d["{0}_per1_all_std".format(var)][4,:])

ax.set_yticks(yval+0.25, labels=file_isf['isf_name'].sel(Nisf=sorted_isf_rignot).values)
#ax.set_yticklabels(file_isf['isf_name'].sel(Nisf=sorted_isf_rignot).values)
ax.set_xlim(2014,2100)

sns.despine()
plt.grid()


In [ ]:
    
    ax.barh(bottom=yval, width=width2, left=start2, height=0.3, color='red',alpha=0.5)#,yerr=d["{0}_per1_all_std".format(var)][4,:])
    ax.barh(bottom=yval+0.3, width=width3, left=start3, height=0.3, color='blue',alpha=0.5)#,yerr=d["{0}_per1_all_std".format(var)][4,:])
    yval = array(range(len(allmodel_list)))
    width2 = end2-start2
    width3 = end3-start3

In [ ]:
for j,var in enumerate(variables_all):
  #for k,season in enumerate(['JFM','AMJ','JAS','OND','year']):
    k=4
    season='year'
    start2=zeros(len(allmodel_list))
    end2=zeros(len(allmodel_list))
    start3=zeros(len(allmodel_list))
    end3=zeros(len(allmodel_list))
    for i,mod in enumerate(range(len(allmodel_list))):
      startP2 = min(d["{0}_mod2_all".format(var)][4,i],d["{0}_mod2_all".format(var)][4,i]+d["{0}_per1_all".format(var)][4,i])
      endP2 = max(d["{0}_mod2_all".format(var)][4,i],d["{0}_mod2_all".format(var)][4,i]+d["{0}_per1_all".format(var)][4,i])
      start2[i]=startP2
      end2[i]=endP2
      startP3 = min(d["{0}_mod2_all".format(var)][4,i]+d["{0}_per1_all".format(var)][4,i],d["{0}_mod2_all".format(var)][4,i]+d["{0}_per1_all".format(var)][4,i]+d["{0}_per2_all".format(var)][4,i])
      endP3 = max(d["{0}_mod2_all".format(var)][4,i]+d["{0}_per1_all".format(var)][4,i],d["{0}_mod2_all".format(var)][4,i]+d["{0}_per1_all".format(var)][4,i]+d["{0}_per2_all".format(var)][4,i])
      start3[i]=startP3
      end3[i]=endP3    
      
    fig, ax = plt.subplots()
    fig1 = matplotlib.pyplot.gcf()
    fig1.set_size_inches(15,22)
    fig1.subplots_adjust(bottom=0.08)
    fig1.subplots_adjust(left=0.18)
    fig1.subplots_adjust(right=0.95)
    fig1.subplots_adjust(top=0.92)
    yval = array(range(len(allmodel_list)))
    width2 = end2-start2
    width3 = end3-start3
    ax.barh(bottom=yval, width=width2, left=start2, height=0.3, color='red',alpha=0.5)#,yerr=d["{0}_per1_all_std".format(var)][4,:])
    ax.barh(bottom=yval+0.3, width=width3, left=start3, height=0.3, color='blue',alpha=0.5)#,yerr=d["{0}_per1_all_std".format(var)][4,:])
    for i,mod in enumerate(allmodel_list):
      plot((d["{0}_mod2_all".format(var)][4,i],d["{0}_mod2_all".format(var)][4,i]), (yval[i],yval[i]+0.6), linewidth=2, color='k')
      plot((d["{0}_mod2_all".format(var)][4,i]-d["{0}_mod2_all_std".format(var)][4,i],d["{0}_mod2_all".format(var)][4,i]+d["{0}_mod2_all_std".format(var)][4,i]), (yval[i]+0.3,yval[i]+0.3), linewidth=2, color='k')
      plot((d["{0}_mod2_all".format(var)][4,i]+d["{0}_per1_all".format(var)][4,i]-d["{0}_per1_all_std".format(var)][4,i],d["{0}_mod2_all".format(var)][4,i]+d["{0}_per1_all".format(var)][4,i]+d["{0}_per1_all_std".format(var)][4,i]), (yval[i]+0.15,yval[i]+0.15), linewidth=2, color='k')
      plot((d["{0}_mod2_all".format(var)][4,i]+d["{0}_per1_all".format(var)][4,i]+d["{0}_per2_all".format(var)][4,i]-d["{0}_per2_all_std".format(var)][4,i],d["{0}_mod2_all".format(var)][4,i]+d["{0}_per1_all".format(var)][4,i]+d["{0}_per2_all".format(var)][4,i]+d["{0}_per2_all_std".format(var)][4,i]), (yval[i]+0.45,yval[i]+0.45), linewidth=2, color='k')
    ax.set_yticks(yval+0.3)
    ax.set_yticklabels( (allmodel_list))
    ax.set_xlabel( lab[var])
    grid()
    plt.show()
    savefig(outputpath+'barchart_float_%s_%s.pdf' %(var,season),rasterize=True)
